
# Assignment 1: Boolean Model, TF-IDF, and Data Retrieval vs. Information Retrieval Conceptual Questions

**Student names**: _Your_names_here_ <br>
**Group number**: _Your_group_here_ <br>
**Date**: _Submission Date_

## Important notes
Please carefully read the following notes and consider them for the assignment delivery. Submissions that do not fulfill these requirements will not be assessed and should be submitted again.
1. You may work in groups of maximum 2 students.
2. The assignment must be delivered in ipynb format.
3. The assignment must be typed. Handwritten assignments are not accepted.

**Due date**: 14.09.2025 23:59

In this assignment, you will:
- Implement a Boolean retrieval model
- Compute TF-IDF vectors for documents
- Run retrieval on queries
- Answer conceptual questions 

---
## Dataset

You will use the **Cranfield** dataset, provided in this file:

- `cran.all.1400`: The document collection (1400 documents)

**The code to parse the file is ready — just update the cran file path to match your own file location. Use the docs variable in your code for the parsed file**

### Load and parse documents (provided)

Run the cell to parse the Cranfield documents. Update the path so it points to your `cran.all.1400` file.


In [1]:

# Read 'cran.all.1400' and parse the documents into a suitable data structure

CRAN_PATH = r"cran.all.1400"  # <-- change this!

def parse_cranfield(path):
    docs = {}
    current_id = None
    current_field = None
    buffers = {"T": [], "A": [], "B": [], "W": []}
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.rstrip("\n")
            if line.startswith(".I "):
                if current_id is not None:
                    docs[current_id] = {
                        "id": current_id,
                        "title": " ".join(buffers["T"]).strip(),
                        "abstract": " ".join(buffers["W"]).strip()
                    }
                current_id = int(line.split()[1])
                buffers = {k: [] for k in buffers}
                current_field = None
            elif line.startswith("."):
                tag = line[1:].strip()
                current_field = tag if tag in buffers else None
            else:
                if current_field is not None:
                    buffers[current_field].append(line)
    if current_id is not None:
        docs[current_id] = {
            "id": current_id,
            "title": " ".join(buffers["T"]).strip(),
            "abstract": " ".join(buffers["W"]).strip()
        }
    print(f"Parsed {len(docs)} documents.")
    return docs

docs = parse_cranfield(CRAN_PATH)



Parsed 1400 documents.


## 1.1 – Boolean Retrieval Model

### 1.1.1 Tokenize documents

Implement tokenization using the given list of stopwords. Create a list of normalized terms per document (e.g., lowercase, remove punctuation/digits; drop stopwords). Store the token lists to use in later steps.

In [2]:
# TODO: Implement tokenization using the given list of stopwords, create list of terms per document

STOPWORDS = set("""a about above after again against all am an and any are aren't as at be because been
before being below between both but by can't cannot could couldn't did didn't do does doesn't doing don't down
during each few for from further had hadn't has hasn't have haven't having he he'd he'll he's her here here's hers
herself him himself his how how's i i'd i'll i'm i've if in into is isn't it it's its itself let's me more most
mustn't my myself no nor not of off on once only or other ought our ours ourselves out over own same shan't she
she'd she'll she's should shouldn't so some such than that that's the their theirs them themselves then there there's
these they they'd they'll they're they've this those through to too under until up very was wasn't we we'd we'll we're
we've were weren't what what's when when's where where's which while who who's whom why why's with won't would wouldn't
you you'd you'll you're you've your yours yourself yourselves""".split())

import re
from typing import Iterable, List

_SANITIZED_STOPWORDS = set()
for w in STOPWORDS:
    _SANITIZED_STOPWORDS.update(re.sub(r"[^a-z]+", " ", w.lower()).split())

def tokenize_documents(docs: Iterable[str]) -> List[List[str]]:
    out = []
    for doc in docs:
        text = f"{doc['id']} {doc['title']} {doc['abstract']}".lower()
        text = re.sub(r"[^a-z]+", " ", doc["abstract"].lower())
        # split and filter
        tokens = [t for t in text.split() if t and t not in _SANITIZED_STOPWORDS]
        out.append(tokens)
    return out

DOC_TOKENS = tokenize_documents(docs.values())
print(docs[1])
print(DOC_TOKENS[1])

{'id': 1, 'title': 'experimental investigation of the aerodynamics of a wing in a slipstream .', 'abstract': 'experimental investigation of the aerodynamics of a wing in a slipstream .   an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem .   the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary-layer-control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory .   an empirical evaluation of the destalling effects was made for the specific configuration

### Build vocabulary

Create a set (or list) of unique terms from all tokenized documents. Report the number of unique terms.


In [7]:
# TODO: Create a set or list of unique terms


# Report: 
# - Number of unique terms

unique_terms = set()
for tokens in DOC_TOKENS:
    unique_terms.update(tokens)

print(f"Number of unique terms: {len(unique_terms)}")

Number of unique terms: 6926


### Build inverted index

For each term, store the list (or set) of document IDs where the term appears.


In [ ]:
appears_in = dict()
# TODO: For each term, store list of document IDs where the term appears
for term in list(unique_terms):
    
    for doc in docs.values():
        if term in doc['title'].lower() or term in doc['abstract'].lower():
            appears_in.update
    print(f"Term '{term}' appears in documents: {appears_in}")

Term 'row' appears in documents: [7, 8, 14, 18, 25, 63, 124, 138, 170, 207, 213, 215, 216, 263, 277, 310, 330, 377, 396, 397, 420, 448, 449, 558, 568, 578, 588, 595, 601, 671, 673, 699, 726, 757, 758, 788, 835, 837, 907, 920, 968, 971, 976, 987, 1025, 1107, 1132, 1142, 1144, 1196, 1313, 1325, 1330, 1333, 1364, 1391]
Term 'rendered' appears in documents: [244, 730]
Term 'decomposed' appears in documents: [942]
Term 'supporting' appears in documents: [1, 100, 239, 506, 762, 766, 858, 1034, 1362]
Term 'theta' appears in documents: [246]
Term 'previous' appears in documents: [52, 62, 77, 79, 80, 81, 139, 185, 195, 201, 210, 212, 244, 264, 266, 276, 297, 320, 325, 334, 365, 375, 396, 400, 448, 449, 455, 501, 502, 527, 540, 554, 558, 564, 579, 580, 586, 601, 611, 617, 619, 633, 642, 644, 645, 680, 694, 697, 733, 739, 740, 782, 814, 820, 822, 826, 829, 830, 831, 833, 838, 843, 866, 871, 872, 889, 893, 917, 928, 932, 933, 937, 940, 989, 1018, 1020, 1028, 1031, 1039, 1041, 1045, 1046, 1051, 105

### Retrieve documents for a Boolean query (AND/OR)

Create a function to retrieve documents for a Boolean query (AND/OR) with query terms.  


In [ ]:
# TODO: Create a function for retrieving documents for a Boolean query (AND/OR) with query terms

from typing import Dict, Iterable, List, Set

def boolean_retrieve(query: str, index: Dict[str, Iterable[int]]) -> List[int]:
    """index: term -> iterable of doc_ids (ints). Returns sorted doc_ids."""
    def postings(term: str) -> Set[int]:
        return set(index.get(term, []))

    tokens = query.split()
    # Split on OR, intersect within each group of ANDs, then union the groups.
    groups: List[List[str]] = []
    cur: List[str] = []
    for tok in tokens:
        u = tok.upper()
        if u == "OR":
            if cur:
                groups.append(cur)
                cur = []
        elif u == "AND":
            continue
        else:
            cur.append(tok.lower())
    if cur:
        groups.append(cur)

    result: Set[int] = set()
    for g in groups:
        if not g:
            continue
        inter = postings(g[0])
        for term in g[1:]:
            inter &= postings(term)
            if not inter:
                break
        result |= inter
    return sorted(result)


In [ ]:
# Do not change this code
boolean_queries = [
  "gas AND pressure",
  "structural AND aeroelastic AND flight AND high AND speed OR aircraft",
  "heat AND conduction AND composite AND slabs",
  "boundary AND layer AND control",
  "compressible AND flow AND nozzle",
  "combustion AND chamber AND injection",
  "laminar AND turbulent AND transition",
  "fatigue AND crack AND growth",
  "wing AND tip AND vortices",
  "propulsion AND efficiency"
]

In [ ]:
# Run Boolean queries in batch, using the function you created
def run_batch_boolean(queries):
    results = {}
    for i, q in enumerate(queries, 1):
        res = boolean_retrieve(q)
        results[f"Q{i}"] = res
    return results

boolean_results = run_batch_boolean(boolean_queries)
for qid, res in boolean_results.items():
    print(qid, "=>", res[:5])


## Part 1.2 – TF-IDF Indexing


$tf_{i,j} = \text{Raw Frequency}$

$idf_t = \log\left(\frac{N}{df_t}\right)$

### Build document–term matrix (TF and IDF weights)

Compute tf and idf using the formulas above and store the weights in a document–term matrix (rows = documents, columns = terms).



In [ ]:
# TODO: Calculate the weights for the documents and the terms using tf and idf weighting. Put these values into a document–term matrix (rows = documents, columns = terms).

# Your code here


### Build TF–IDF document vectors

From the matrix, build a TF–IDF vector for each document (consider normalization if needed for cosine similarity).


In [ ]:

# TODO: Build TF–IDF document vectors from the document–term matrix
# Your code here


### Implement cosine similarity

Implement a function to compute cosine similarity scores between a (tokenized) query and all documents.


In [ ]:

# TODO: Create a function for calculating the similarity score of all the documents by their relevance to query terms

def tfidf_retrieve(query: str):
    # Your code here
    raise NotImplementedError("Implement TF-IDF retrieval here")


In [ ]:
# Do not change this code
tfidf_queries = [
  "gas pressure",
  "structural aeroelastic flight high speed aircraft",
  "heat conduction composite slabs",
  "boundary layer control",
  "compressible flow nozzle",
  "combustion chamber injection",
  "laminar turbulent transition",
  "fatigue crack growth",
  "wing tip vortices",
  "propulsion efficiency"
]

In [ ]:
# Run TF-IDF queries in batch (print top-5 results for each), using the function you created
def run_batch_tfidf(queries):
    results = {}
    for i, q in enumerate(queries, 1):
        res = tfidf_retrieve(q)
        results[f"Q{i}"] = res
    return results

tfidf_results = run_batch_tfidf(tfidf_queries)

for qid, res in tfidf_results.items():
    print(qid, "=>", res[:5])



## Part 1.3 – Conceptual Questions

Answer the following questions:

**1. What is the difference between data retrieval and information retrieval?**
*Your answer here*

**For the following scenarios, which approach would be suitable data retrieval or information retrieval? Explain your reasoning.** <br>
1.a A clerk in pharmacy uses the following query: Medicine_name = Ibuprofen_400mg
*Your answer here*

1.b A clerk in pharmacy uses the following query: An anti-biotic medicine 
*Your answer here*

1.c Searching for the schedule of a flight using the following query: Flight_ID = ZEFV2
*Your answer here*

1.d Searching an E-commerce website using the following query to find an specific shoe: Brooks Ghost 15
*Your answer here*

1.e Searching the same E-commerce website using the following query: Nice running shoes
*Your answer here*
